---
title: "Data Cleaning"
format:
    html: 
        code-fold: false
---

<!-- After digesting the instructions, you can delete this cell, these are assignment instructions and do not need to be included in your final submission.  -->

{{< include instructions.qmd >}} 

## Code 

### Importing Packages

First, let's import the necessary packages. 

In [1]:
# Packages
import gzip # For unzipping the raw data
import pandas as pd # Using pandas for easier data manipulation
import nltk # Using nltk for its list of stopwords
import string # string.punctuation will be used for text cleaning

import warnings # Turning off warnings for cleaner output
warnings.filterwarnings('ignore')



### Loading raw data

Now we can load in the processed CSV we constructed in the previous section.

In [2]:
# Pathway to raw data
data_path = "../../data/raw-data/book_reviews.csv.gz"

# Unzip the CSV file
with gzip.open(data_path, 'rb') as f:
    # Read the CSV file into a dataframe
    reviews_raw = pd.read_csv(f)

# Display the first few lines
reviews_raw.head()

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image
0,5.0,67,True,"09 18, 1999",AAP7PPBU72QFM,0151004714,{'Format:': ' Hardcover'},D. C. Carrad,This is the best novel I have read in 2 or 3 y...,A star is born,937612800,NaN
1,3.0,5,True,"10 23, 2013",A2E168DTVGE6SV,0151004714,{'Format:': ' Kindle Edition'},Evy,"Pages and pages of introspection, in the style...",A stream of consciousness novel,1382486400,NaN
2,5.0,4,False,"09 2, 2008",A1ER5AYS3FQ9O3,0151004714,{'Format:': ' Paperback'},Kcorn,This is the kind of novel to read when you hav...,I'm a huge fan of the author and this one did ...,1220313600,NaN
3,5.0,13,False,"09 4, 2000",A1T17LMQABMBN5,0151004714,{'Format:': ' Hardcover'},Caf Girl Writes,What gorgeous language! What an incredible wri...,The most beautiful book I have ever read!,968025600,NaN
4,3.0,8,True,"02 4, 2000",A3QHJ0FXK33OBE,0151004714,{'Format:': ' Hardcover'},W. Shane Schmidt,I was taken in by reviews that compared this b...,A dissenting view--In part.,949622400,NaN


Here is our first look at the raw dataframe - we can observe 12 columns containing the following variables:

- `overall`: Rating 
- `vote`: Amount of community votes given to the review. Users will often vote when they find a review helpful
- `verified`: Boolean variable that indicates whether an account is verified or not
- `reviewTime`: Time of the review (raw)
- `reviewerID`: ID of the reviewer
- `asin`: Id of the product
- `style`: Key-value object. In this case, it describes the format of the book (e.g. Paperback, Hardcover, Kindle, etc.)
- `reviewerName`: Name of the reviewer
- `reviewText`: Raw, unprocessed text contents of the review
- `summary`: The title of the user's review
- `unixReviewTime`: Time of the review in unix time (Measures time "based by the number of non-leap seconds that have elapsed since 00:00:00 UTC")[@UnixTime].
- `image`: Image path (if any)

### Dropping Unnecessary Columns

First, lets drop the `unixReviewTime` and `image` columns, since they will not be used in our analysis

In [3]:
# list of columns to drop
drop_cols = ['unixReviewTime', 'image']

# dropping columns
reviews_raw = reviews_raw.drop(columns=drop_cols)

# ensuring correct columns were dropped
print(reviews_raw.columns)

Index(['overall', 'vote', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'style', 'reviewerName', 'reviewText', 'summary'],
      dtype='object')


### Checking for missing values

Now lets check to see if our dataset contains any NA values

In [4]:
print(reviews_raw.isnull().sum())

overall               0
vote            5790916
verified              0
reviewTime            0
reviewerID            0
asin                  0
style           3192170
reviewerName       1472
reviewText         1640
summary             875
dtype: int64


It looks like our most important column, `reviewText`, has 1640 null values, so let's get rid of them.

In [6]:
# Taking only rows or reviews_raw where reviewText is not null
reviews_raw = reviews_raw[reviews_raw['reviewText'].notna()]

# Lets make sure this works
print(f"Number of NULL values in reviewText Column: {reviews_raw['reviewText'].isnull().sum()}")

Number of NULL values in reviewText Column: 0


### Checking data types

Before moving forward, let's ensure that each of our column data types are appropriate

In [8]:
print(reviews_raw.dtypes)

overall         float64
vote             object
verified           bool
reviewTime       object
reviewerID       object
asin             object
style            object
reviewerName     object
reviewText       object
summary          object
dtype: object


Lets make the following changes:

- Convert `vote` to an int object, where NA values are replaced by 0

In [13]:
# Converting 'vote' to integer, while dropping columns from strings and replacing NA with 0 
reviews_raw['vote'] = reviews_raw['vote'].replace({',': ''}, regex=True).fillna(0).astype(int)

print(f"Vote column data type: {reviews_raw['vote'].dtype}")
print(f"Number of NULL values in vote Column: {reviews_raw['vote'].isnull().sum()}")

Vote column data type: int64
Number of NULL values in vote Column: 0


array(["{'Format:': ' Hardcover'}", "{'Format:': ' Kindle Edition'}",
       "{'Format:': ' Paperback'}", ...,
       '{\'Size:\': \' 5/16" - 5FT\', \'Color:\': \' Red\'}',
       '{\'Size:\': \' 1" - 3FT\', \'Color:\': \' Red\'}',
       '{\'Size:\': \' 3/4" - 5FT\', \'Color:\': \' Red\'}'], dtype=object)